In [ ]:
!pip install sentencepiece
# !wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt

     |████████████████████████████████| 1.1MB 8.5MB/s 
--2020-11-25 12:13:21--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt’

botchan.txt         100%[===================>] 272.25K  --.-KB/s    in 0.02s   

2020-11-25 12:13:21 (12.1 MB/s) - ‘botchan.txt’ saved [278779/278779]



In [ ]:
import gc
import os
import random

import numpy as np
import sentencepiece as spm
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from tqdm import tqdm


class SortedBatchSampler:
    def __init__(self, data, batch_size, shuffle, sort_key):
        self.shuffle = shuffle

        print("Sorting....")
        zip_ = []
        for i, row in tqdm(enumerate(data), total=len(data)):
            zip_.append(tuple([i, sort_key(row)]))
        zip_ = sorted(zip_, key=lambda r: r[1])
        indexes = [item[0] for item in zip_]

        self.batches = np.array_split(indexes, len(indexes) // batch_size)

    def __iter__(self):
        if self.shuffle:
            random.shuffle(self.batches)
        return iter(self.batches)

    def __len__(self):
        return len(self.batches)


class TextProcessor:
    def __init__(self, sub_dim, training_file="", prefix="subwords", model_type="bpe"):
        super().__init__()
        self.prefix = prefix + "_" + str(sub_dim) + "_" + model_type
        # training_file = ""
        if not os.path.isfile(self.prefix + ".model"):
            spm.SentencePieceTrainer.Train('--input=' + training_file +
                                           ' --model_prefix=' + self.prefix +
                                           ' --character_coverage=1.0' +
                                           ' --vocab_size=' + str(sub_dim) +
                                           ' --model_type=' + model_type +
                                           ' --max_sentence_length=100000' +
                                           ' --split_by_whitespace=true')

        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(self.prefix + ".model")

    def encode(self, text):
        return self.sp.EncodeAsIds(text)

    def decode(self, id_array):
        return self.sp.decode_ids(id_array)


class GPT2Dataset(Dataset):
    def __init__(self, corpus_path, max_len, n_tokens, dataset_plk_path="data/gpt2_dataset.npy"):

        self.max_len = max_len
        if os.path.isfile(dataset_plk_path):
            self.corpus = np.load(dataset_plk_path, allow_pickle=True)
        else:
            self.corpus = []
            processor = TextProcessor(n_tokens, corpus_path)
            print("Encoding texts...")
            with open(corpus_path, 'r') as file:
                for line in tqdm(file):
                    encoded = processor.encode(line)
                    self.corpus.append(np.array(encoded, dtype=np.int16))
            gc.collect()
            np.save(dataset_plk_path, self.corpus)

    def __getitem__(self, index):
        if index > self.__len__():
            print(index)
            raise IndexError()

        encoded = self.corpus[index]
        offset = random.randint(0, max(0, len(encoded) - self.max_len))
        encoded = encoded[offset:self.max_len + offset]

        return torch.LongTensor(encoded)

    def __len__(self):
        return len(self.corpus)


def collate(batch):
    return pad_sequence(batch, batch_first=True, padding_value=0)


# Model GPT-2


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import kaiming_normal_


class ScaledDotProductAttention(nn.Module):
    def __init__(self, key_dim, drop=0.1):
        super().__init__()
        self.temperature = np.power(key_dim, 0.5)
        self.dropout = nn.Dropout(drop)

    def forward(self, q, k, v):
        energies = (torch.bmm(q, k.transpose(1, 2))) / self.temperature

        seq_len = energies.size(1)
        mask = (torch.tril(torch.ones(seq_len, seq_len)) == 0).to(energies.device)
        energies.masked_fill_(mask, -np.inf)

        attention = self.dropout(F.softmax(energies, dim=2))
        context = torch.bmm(attention, v).squeeze(1)
        return context, attention


class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = num_heads
        self.attn = ScaledDotProductAttention(embed_dim)

        self.query_trans = nn.Linear(embed_dim, embed_dim)
        self.keys_trans = nn.Linear(embed_dim, embed_dim)
        self.value_trans = nn.Linear(embed_dim, embed_dim)
        self.projection = nn.Linear(embed_dim, embed_dim)
        kaiming_normal_(self.query_trans.weight, nonlinearity="linear")
        kaiming_normal_(self.keys_trans.weight, nonlinearity="linear")
        kaiming_normal_(self.value_trans.weight, nonlinearity="linear")
        kaiming_normal_(self.projection.weight, nonlinearity="linear")

        self.keys_trans = self.keys_trans.half()
        self.value_trans = self.value_trans.half()
        self.projection = self.projection.half()

    def split_heads(self, x):
        bs, seq_len, _ = x.size()
        # result: (head*batch_size) x seq_len x new features
        return x.view(bs, seq_len, self.n_heads, -1).permute(2, 0, 1, 3).reshape(self.n_heads * bs, seq_len, -1)

    def merge_heads(self, x):
        _, seq_len, features_size = x.size()
        x = x.view(self.n_heads, -1, seq_len, features_size)
        bs = x.size(1)
        # batch_size x seq_len x heads x features
        return x.permute(1, 2, 0, 3).reshape(bs, seq_len, -1)

    def forward(self, x):
        q = self.split_heads(self.query_trans(x))
        k = self.split_heads(self.keys_trans(x.half()).float())
        v = self.split_heads(self.value_trans(x.half()).float())
        a, _ = self.attn(q, k, v)
        a = self.merge_heads(a)
        return self.projection(a.half())


class MLP(nn.Module):
    def __init__(self, embed_dim, factor=4):
        super(MLP, self).__init__()
        self.fc = nn.Linear(embed_dim, embed_dim * factor)
        self.fc2 = nn.Linear(embed_dim * factor, embed_dim)

        kaiming_normal_(self.fc.weight, nonlinearity="relu")
        kaiming_normal_(self.fc2.weight, nonlinearity="linear")

        self.fc = self.fc.half()
        self.fc2 = self.fc2.half()

    def forward(self, x):
        h = F.gelu(self.fc(x.half()).float())
        return self.fc2(h.half())


class Block(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(Block, self).__init__()
        self.ln_1 = nn.LayerNorm(embed_dim)
        self.attn = MultiHeadAttention(embed_dim, num_heads)
        self.ln_2 = nn.LayerNorm(embed_dim)
        self.mlp = MLP(embed_dim)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


def get_positional_encoding(n_positions, n_embd):
    def angle_defn(pos, i, d_model_size):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model_size))
        return pos * angle_rates

    angle_rads = angle_defn(np.arange(n_positions)[:, np.newaxis], np.arange(n_embd)[np.newaxis, :], n_embd)
    sines = np.sin(angle_rads[:, 0::2])
    cosines = np.cos(angle_rads[:, 1::2])
    pos_encoding = torch.tensor(np.concatenate([sines, cosines], axis=-1)[np.newaxis, ...], dtype=torch.float)
    return pos_encoding


class GPT2Model(nn.Module):
    def __init__(self):
        super(GPT2Model, self).__init__()
        self.n_layer = 12
        self.n_embd = 768 + 128
        self.n_head = 12 + 2
        self.n_tokens = 512
        self.n_positions = 512  # TODO

        self.wte = nn.Embedding(self.n_tokens, self.n_embd, padding_idx=0).half()
        self.register_buffer('positional_encoding', get_positional_encoding(self.n_positions, self.n_embd))
        self.blocks = nn.ModuleList([Block(self.n_embd, self.n_head) for _ in range(self.n_layer)])

        self.decoder_norm = nn.LayerNorm(self.n_embd)
        self.decoder = nn.Linear(self.n_embd, self.n_tokens, bias=False)
        kaiming_normal_(self.decoder.weight, nonlinearity="linear")

    def forward(self, input_ids):
        inputs_embeds = self.wte(input_ids)
        position_embeds = self.positional_encoding[:, :input_ids.size(1), :]
        hidden_states = (inputs_embeds + position_embeds).float()

        for block in self.blocks:
            hidden_states = block(hidden_states)

        decoded = self.decoder(self.decoder_norm(hidden_states))
        return F.log_softmax(decoded, dim=-1)


# Train



In [ ]:
! git clone https://github.com/NVIDIA/apex.git
% cd apex
% cd fp16_utils
!pip install --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ../..

fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
[Errno 2] No such file or directory: 'fp16_utils'
/content/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Processing /content/apex
Skipping wheel build for apex, due to binaries being disabled for it.
    Running setup.py install for apex ... done
/


In [ ]:
import torch.optim as optim
from torch.optim import SGD

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
root = "/gdrive/My Drive/CS470"
import os
os.chdir(root)
os.getcwd()

'/gdrive/My Drive/CS470'

In [ ]:
import torch
from apex.fp16_utils import FP16_Optimizer
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

# from lm.gpt2.data.data_loader import GPT2Dataset, collate, SortedBatchSampler
# from lm.gpt2.lamb import Lamb
# from lm.gpt2.model import GPT2Model

if __name__ == '__main__':
    # lr = 0.0001
    lr = 1e-7
    wd = 1.2e-6
    batch_size = 8
    load_ver = 0
    device = "cuda:0"
    corpus_path = "data/processed_lyrics.txt"

    model = GPT2Model()
    model = model.to(device)
    dataset = GPT2Dataset(corpus_path, model.n_positions + 1, model.n_tokens)
    sampler = SortedBatchSampler(dataset, batch_size, True, sort_key=lambda d: d.size(0))
    train_loader = DataLoader(dataset, batch_sampler=sampler, collate_fn=collate, num_workers=1, pin_memory=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
            factor=0.25, patience=1, threshold=0.0001, threshold_mode='rel', 
            cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    # optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    # optimizer = FP16_Optimizer(optimizerr, dynamic_loss_scale=True)
    # criterion = torch.nn.NLLLoss(ignore_index=0).to(device)

    criterion = nn.CrossEntropyLoss() 

    print("\n\nTotal params: " + str(sum(p.numel() for p in model.parameters())))

    if load_ver != 0:
        model.load_state_dict(torch.load(f"models/model{load_ver}", map_location=device))
        optimizer.load_state_dict(torch.load(f"models/optimizer_state", map_location=device))

    running_loss = 0
    # for iteration in range(1 + load_ver, 30):
    for iteration in range(1):
        print("\n\n\nIteration: " + str(iteration))
        
        for i, contexts in enumerate(tqdm(train_loader, smoothing=0)):
            contexts = contexts.to(device)
            # if (len(contexts) > 0 and contexts.shape[0] != 0 and contexts.shape[1]):
            [a, b] = list(contexts.size())
            # print(contexts)
            input_x = contexts[:, :-1]
            # input_x = contexts.contiguous().view(a, 1 if b==0 else -1)
            # 
            model.train()
            if (input_x.shape[1] > 1):
              optimizer.zero_grad()
              # print('input', input_x)
              targets = contexts[:, 1:]
              
              # print('nan', torch.isnan(targets))
              # print('inf', torch.isinf(targets))
              # targets = contexts[:, (1 if b!=0 else 0):].contiguous().view(a, 1 if (b-1)==0 else -1)
              # print('target', targets.shape)
              
              model = GPT2Model() 
              model = model.to(device)
              output = model(input_x)
              # print('output', output)
              loss = criterion(output.view(output.shape[0] * output.shape[1], -1), targets.reshape(-1))
              # optimizer.backward(loss)
              
              loss.backward()
              # optimizer.clip_master_grads(5)
              torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
              optimizer.step()

              # Loss printer
              loss_value = loss.item()
              print('loss', loss_value)
              running_loss = loss_value if running_loss == 0 else (running_loss * 0.99 + loss_value * 0.01)
              if i % 100 == 0:
                  print(f"Loss: {running_loss} Current loss: {loss_value}")

              # Partial saver
              if i > 0 and i % 500 == 0:
                  print("\n\nPartial save...")
                  torch.save(model.state_dict(), f"models/model_partial")
                  torch.save(optimizer.state_dict(), f"models/optimizer_state_partial")

        scheduler.step(running_loss)

        print("\n\nSave model...")
        torch.save(model.state_dict(), f"models/model" + str(iteration))
        torch.save(optimizer.state_dict(), f"models/optimizer_state")
















  0%|          | 0/15173 [00:00<?, ?it/s]













 29%|██▉       | 4426/15173 [00:00<00:00, 44253.08it/s]

Sorting....
















 58%|█████▊    | 8773/15173 [00:00<00:00, 44013.53it/s]













100%|██████████| 15173/15173 [00:00<00:00, 43616.62it/s]














  0%|          | 0/1896 [00:00<?, ?it/s]



Total params: 116664576



Iteration: 0
















  0%|          | 1/1896 [00:02<1:33:44,  2.97s/it]

...

loss 6.503606796264648
loss 6.916902542114258
















 80%|████████  | 1519/1896 [1:04:31<16:00,  2.55s/it]

loss 6.721122741699219
loss 6.760342121124268
















 80%|████████  | 1521/1896 [1:04:36<15:55,  2.55s/it]

loss 6.485570907592773



# Generate

In [ ]:
def generate(start_sent, max_tokens=300, temperature=0.7, top_k=32):
    start_sent = text_processor.encode(start_sent)

    with torch.no_grad():
        for i in range(max_tokens):
            input_ids = torch.LongTensor(start_sent).unsqueeze(0).to(device)
            output = model(input_ids)

            word_weights = output.squeeze()[-1].div(temperature).exp().cpu()
            tops = word_weights.topk(top_k)
            word_idx = torch.multinomial(tops[0], 1)[0].item()
            word_idx = tops[1][word_idx].item()

            start_sent.append(word_idx)

    decoded = text_processor.decode(start_sent)
    # print(decoded)

if __name__ == '__main__':
    device = "cuda:0"
    saved_model_path = "models/model_partial"
    start_sentence = "opel"
    # corpus_path = "data/processed_lyrics.txt"
    model = GPT2Model().to(device)
    text_processor = TextProcessor(model.n_tokens, "data/processed_lyrics.txt",prefix="")
    model.load_state_dict(torch.load(saved_model_path, map_location=device))
    model.eval()

    generate(start_sentence)

RuntimeError: ignored